In [1]:
# Importing necessary libraries
# TODO:clean this
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import keras
from tensorflow import keras
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import categorical_crossentropy
# Importing necessary libraries
import pandas as pd
import numpy as np


from matplotlib import pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'


# from transformers import SentenceTransformer
import numpy as np
import json
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from collections import OrderedDict


import pyarrow.parquet as pa

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


2024-05-28 15:35:16.695246: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 15:35:16.806785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 15:35:19.006185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_captions.csv', index_col = [0])
df

,mp3_path,tags,caption_writing
clip_id,,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera,Experience the majestic beauty of classical mu...
6,american_bach_soloists-j_s__bach_solo_cantatas...,opera,Experience the rich sound of classical eleganc...
10,american_bach_soloists-j_s__bach_solo_cantatas...,opera,This powerful classic opera piece showcases th...
11,american_bach_soloists-j_s__bach_solo_cantatas...,opera,This atmospheric and introspective song blends...
14,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic,This upbeat dance track features a pulsing tec...
...,...,...,...
58896,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This classical guitar solo piece features intr...
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This beautiful classical piece features a haun...
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This classical piece features beautiful melodi...


In [3]:
# Define the Encoder class focusing only on the encoding part
class Encoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Encoder, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, encoding_size),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.layers(x)
    
# Define input and encoding sizes
input_size = 768
encoding_size = 64
def load_encoder_state_dict(encoder, state_dict_path):
    state_dict = torch.load(state_dict_path)
    # Rename the state_dict keys to match the structure of the Encoder class
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        # Prepend 'layers.' to each key
        name = f'layers.{k}'
        new_state_dict[name] = v
    
    encoder.load_state_dict(new_state_dict)
    return encoder

def get_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device = get_device()
# Instantiate the encoder
encoder = Encoder(input_size=input_size, encoding_size=encoding_size).to(device)
encoder.eval()  # Set the encoder to evaluation mode

# Load the pre-trained weights for the encoder
encoder = load_encoder_state_dict(encoder, '/home/mendu/Thesis/data/musiccaps/auto_encoder/encoder_state_dict.pth')

# Load the SentenceTransformer model and move to the correct device
roberta_model = SentenceTransformer('/home/mendu/Thesis/data/musiccaps/new_embedding_model').to(device)

# Assuming df is a predefined DataFrame with text data
caption = df.caption_writing[2]
# Function to encode caption
def encode_caption(encoder, sentence_model, text):
    with torch.no_grad():
        encoded_caption = sentence_model.encode(text, convert_to_tensor=True)
        encoded_caption = encoded_caption.to(device)
        return encoder(encoded_caption.unsqueeze(0))
    
# Encoded caption
fully_encoded_caption = encode_caption(encoder, roberta_model, caption)

In [4]:
fully_encoded_caption

tensor([[ 52.0009,  28.6403,  34.2275,  59.2607,   1.1570,  57.7524, 142.9124,
          44.6702,  41.6169, 104.4225,  92.6284, 119.1738,  42.9483,  79.1854,
         216.7657,  67.0605,  43.6379,  77.6768, 100.6521,  87.6473, 132.7112,
         126.8025,  86.1116, 131.3719,  68.6272,  81.4069, 134.8302, 196.9826,
          90.1404,  65.5783,  97.3774, 187.2035,  53.5716, 243.3475,   0.0000,
           0.0000,  55.3763, 132.5184,  23.2223,   0.0000, 155.0516,  99.4067,
          38.8811, 111.8058, 213.8127, 153.5288,   0.0000,  39.6156,  19.5320,
         135.4064,  96.3223,  40.3084, 116.1382, 154.0468,  79.5539,  27.2223,
          96.1176,  39.1434, 160.7672,  63.0411,  12.1961,  99.3322,  49.6889,
          55.5985]], device='cuda:0')

In [5]:
# Apply the encode_caption function to each row's caption_writing
def get_encoded_caption(row):
    caption = row['caption_writing']
    return encode_caption(encoder, roberta_model, caption).cpu().clone().numpy().tolist()


df['caption_embedding'] = df.apply(get_encoded_caption, axis=1)

# Flatten the nested list
df['caption_embedding'] = df['caption_embedding'].apply(lambda x: x[0])

# Now df has an additional column with the caption embeddings as tensors
df

,mp3_path,tags,caption_writing,caption_embedding
clip_id,,,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera,Experience the majestic beauty of classical mu...,"[52.000892639160156, 28.640304565429688, 34.22..."
6,american_bach_soloists-j_s__bach_solo_cantatas...,opera,Experience the rich sound of classical eleganc...,"[41.24440002441406, 41.702205657958984, 49.210..."
10,american_bach_soloists-j_s__bach_solo_cantatas...,opera,This powerful classic opera piece showcases th...,"[49.760990142822266, 39.03318786621094, 39.876..."
11,american_bach_soloists-j_s__bach_solo_cantatas...,opera,This atmospheric and introspective song blends...,"[74.27188110351562, 52.33830261230469, 113.251..."
14,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic,This upbeat dance track features a pulsing tec...,"[37.00017547607422, 0.0, 132.0074920654297, 71..."
...,...,...,...,...
58896,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This classical guitar solo piece features intr...,"[94.79190826416016, 0.0, 19.968608856201172, 7..."
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This beautiful classical piece features a haun...,"[74.07515716552734, 20.25941276550293, 33.8745..."
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This classical piece features beautiful melodi...,"[76.3675308227539, 31.28574562072754, 26.47723..."


In [6]:
df

,mp3_path,tags,caption_writing,caption_embedding
clip_id,,,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera,Experience the majestic beauty of classical mu...,"[52.000892639160156, 28.640304565429688, 34.22..."
6,american_bach_soloists-j_s__bach_solo_cantatas...,opera,Experience the rich sound of classical eleganc...,"[41.24440002441406, 41.702205657958984, 49.210..."
10,american_bach_soloists-j_s__bach_solo_cantatas...,opera,This powerful classic opera piece showcases th...,"[49.760990142822266, 39.03318786621094, 39.876..."
11,american_bach_soloists-j_s__bach_solo_cantatas...,opera,This atmospheric and introspective song blends...,"[74.27188110351562, 52.33830261230469, 113.251..."
14,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic,This upbeat dance track features a pulsing tec...,"[37.00017547607422, 0.0, 132.0074920654297, 71..."
...,...,...,...,...
58896,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This classical guitar solo piece features intr...,"[94.79190826416016, 0.0, 19.968608856201172, 7..."
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This beautiful classical piece features a haun...,"[74.07515716552734, 20.25941276550293, 33.8745..."
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical,This classical piece features beautiful melodi...,"[76.3675308227539, 31.28574562072754, 26.47723..."


In [ ]:
# Save to CSV
# df.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_embeddings.csv', index=False)

## Training the model

In [7]:
def add_gauss_noise(image, mean = 0, var = 10):
    '''function to add gaussian noise to an image'''
    sigma = var ** 0.5

    #creating a noise image with gaussian distribution
    gaussian = np.random.normal(mean, sigma, (image.shape[0], image.shape[1]))

    #creating a zeroes image
    noisy_image = np.zeros(image.shape, np.float32)

    #adding the noise to the original image
    noisy_image = image + gaussian

    #normalising the image
    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)

    #changing type
    noisy_image = noisy_image.astype(np.uint8)

    return noisy_image

In [8]:
#loading the cropped .npy files in a tensor

#the folder path to where the mel-specs are stored
folder_path_mel = '/home/mendu/Thesis/data/magnatagatune/mel-specs'

#The ordered list of all the filenames in the filtered_df
audio_names_list = df[:500].mp3_path.to_list()

In [9]:
df[:500].tags.value_counts()

tags
electronic    175
rock           91
classical      86
indian         52
opera          47
new age        27
pop            12
country        10
Name: count, dtype: int64

In [10]:
len(audio_names_list)

500

In [11]:
data = []
i = 0

for filename in audio_names_list: #iterates over the filtered_df audio names
  filename_npy = str(filename[:-4]) + '.npy'
  file_path = os.path.join(folder_path_mel, filename_npy) #creates a folder path for the mel_specs 
  img = np.load(file_path)
  data.append(add_gauss_noise(img))

  # To ensure that the files are getting loaded in the correct order
  if i < 10 :
    print(filename)
  i = i+1

data = np.array(data)

american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-30-59.mp3
american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-146-175.mp3
american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-262-291.mp3
american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-291-320.mp3
lvx_nova-lvx_nova-01-contimune-30-59.mp3
american_bach_soloists-j_s__bach__cantatas_volume_v-01-gleichwie_der_regen_und_schnee_vom_himmel_fallt_bwv_18_i_sinfonia-117-146.mp3
steven_devine-portrait_of_an_english_harpsichord-01-lesson_1_in_g_major_prelude_james_nares-30-59.mp3
the_headroom_project-jetuton_andawai-01-linda_morena-88-117.mp3
american_bach_soloists-heinrich_schutz__musicalische_exequien-01-musicalische_exequien_swv_279_teil_i_concert_in_form_einer_teutschen_begrabnismissa-30-59.mp3
american_bach_soloists-heinrich_schutz__musicalische_exequien-01-musicalische_exequien_swv_279_teil_i_concert_in_form_einer_teutschen_begrabnismissa-146-175.mp3


In [12]:
# Caption Embeddings
embeddings = (df['caption_embedding'][:500])

In [13]:
len(embeddings)

500

In [14]:
# data = np.load('data/magnatagatune/data_array.npy')

In [15]:
#normalize data
data = data.astype('float32')/255.0

data.shape

(500, 128, 1255)

In [16]:
# Reshape data to include the channel dimension
data = data.reshape((-1, 128, 1255, 1))

# Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(data, pd.get_dummies(df[:500]['tags']), test_size=0.1, random_state=42)
# Split data into training and validation sets
X_train_spec, X_val_spec, X_train_embed, X_val_embed, y_train, y_val = train_test_split(data, embeddings, pd.get_dummies(df[:500]['tags']), test_size=0.1, random_state=42)

In [17]:
data.shape

(500, 128, 1255, 1)

In [18]:
# Initialize ImageDataGenerator with necessary augmentations
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2
)

datagen.fit(X_train_spec)

In [19]:
# Define num_classes based on the number of unique genres
num_classes = y_train.shape[1]
num_classes

8

In [25]:
class ModelSubClassing(keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        # Layer of Block 1
        self.conv1 = Conv2D(32, 3, strides=2, activation="relu")
        self.max1  = MaxPooling2D(3)
        self.bn1   = BatchNormalization()

        # Layer of Block 2
        self.conv2 = Conv2D(64, 3, activation="relu")
        self.bn2   = BatchNormalization()
        self.max2  = MaxPooling2D(3)
        self.drop1 = Dropout(0.3)
        
        # Layer of Block 3
        self.conv3 = Conv2D(128, 3, activation="relu")
        self.bn3   = BatchNormalization()
        self.max3  = MaxPooling2D(3)
        self.drop2 = Dropout(0.3)

        # Flatten, followed by 3 dense layers: 128, 64, num_classes
        self.flatten   = Flatten()
        self.dense128 = Dense(128, activation="relu")
        self.dense64 = Dense(64, activation="relu")
        self.dense_out = Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        # Unpack the inputs
        mel_spectrogram, caption_embedding = inputs

        # Forward pass: block 1
        x = self.conv1(mel_spectrogram)
        x = self.max1(x)
        x = self.bn1(x)

        # Forward pass: block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.max2(x)
        x = self.drop1(x, training=training)

        # Forward pass: block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.max3(x)
        x = self.drop2(x, training=training)

        # Flatten and dense layers
        x = self.flatten(x)
        x = self.dense128(x)
        x_dense64 = self.dense64(x)
        y_hat = self.dense_out(x_dense64)
        x_dense64_list = x_dense64.cpu().clone().numpy().tolist()
        
        return y_hat, x_dense64_list

    def train_step(self, data):
        # Unpack the data
        (mel_spectrogram, caption_embedding), y = data

        with tf.GradientTape() as tape:
            y_pred, x_dense64 = self((mel_spectrogram, caption_embedding), training=True)  # Forward pass
            # Compute the standard loss value
            loss = self.compiled_loss(y, y_pred)
            # Add the custom loss component
            custom_loss = tf.reduce_mean(tf.square(caption_embedding - x_dense64))
            total_loss = loss + custom_loss

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

# Example of using the custom model
# num_classes = 10  # Example number of output classes
model = ModelSubClassing(num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Create some dummy data
# import numpy as np
# X_mel = np.random.random((10, 128, 1255, 1))
# X_caption = np.random.random((10, 64))
# y = np.random.randint(0, num_classes, 10)

# Train the model
# model.fit((X_mel, X_caption), y, epochs=3, batch_size=2)

2024-05-28 16:00:20.174645: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [26]:
# Use the early stopping callback to halt training if validation loss doesn't improve
early_stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Fit the model
history = model.fit(
    (X_train_spec, X_train_embed),
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=((X_val_spec, X_val_embed), y_val),
    callbacks=[early_stopper]
)

# history = model.fit((X_train_spec, X_train_embed), y_train, epochs=..., batch_size=..., validation_data=((X_val_spec, X_val_embed), y_val), ...)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [14]:
class ModelSubClassing(keras.Model):
    def __init__(self, num_classes, dim):
        super().__init__()
        # define all layers in init
        # Layer of Block 1
        self.conv1 = keras.layers.Conv2D(
                          32, 3, strides=2, activation="relu"
                     )
        self.max1  = keras.layers.MaxPooling2D(3)
        self.bn1   = keras.layers.BatchNormalization()

        # Layer of Block 2
        self.conv2 = keras.layers.Conv2D(64, 3, activation="relu")
        self.bn2   = keras.layers.BatchNormalization()
        self.max2  = keras.layers.MaxPooling2D(3)
        self.drop  = keras.layers.Dropout(0.3)
        
        # Layer of Block 3
        self.conv3 = keras.layers.Conv2D(128, 3, activation="relu")
        self.bn3   = keras.layers.BatchNormalization()
        self.max3  = keras.layers.MaxPooling2D(3)
        self.drop  = keras.layers.Dropout(0.3)

        # flatten, followed by 3 dense layers 128, 64, 8
        self.flatten   = keras.layers.Flatten()
        self.dense128 = keras.layers.Dense(128)
        self.dense64 = keras.layers.Dense(64)
        self.dense8 = keras.layers.Dense(num_classes)


    def call(self, input_tensor, training=False):
        # forward pass: block 1 
        x = self.conv1(input_tensor)
        x = self.max1(x)
        x = self.bn1(x)

        # forward pass: block 2 
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.max2(x)
        
        # forward pass: block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.max3(x)

        # droput followed by flatten and dense layers
        x = self.drop(x)
        x = self.flatten(x)
        x = self.dense128(x)
        x_dense64 = self.dense64(x)
        y_hat = self.dense8(x_dense64)
        return y_hat
    
    def build_graph(self):
        x = Input(shape=(dim))
        return Model(inputs=[x], outputs=self.call(x))

In [15]:
dim = (128, 1255, 1)

In [16]:
sub_classing_model = ModelSubClassing(num_classes, dim)
sub_classing_model.build((None, *dim))
sub_classing_model.build_graph().summary()

2024-05-28 10:32:31.527853: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/mendu/miniconda3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/layer.py:359: UserWarning: `build()` was called on layer 'model_sub_classing', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 1255, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 63, 627, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 21, 209, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 21, 209, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 19, 207, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 19, 207, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 69, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 67, 128)     │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 67, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 22, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 22, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2816)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       360,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 462,920 (1.77 MB)

 Trainable params: 462,472 (1.76 MB)

 Non-trainable params: 448 (1.75 KB)

In [17]:
sub_classing_model.compile(
          loss      = keras.losses.CategoricalCrossentropy(),
          metrics   = ['accuracy'],
          optimizer = keras.optimizers.Adam())

In [18]:
# fit 
sub_classing_model.fit(X_train, y_train, batch_size=128, epochs=1)

ValueError: For a model with multiple outputs, when providing the `metrics` argument as a list, it should have as many entries as the model has outputs. Received:
metrics=['accuracy']
of length 1 whereas the model has 2 outputs.

To create a custom model class in Keras with the specified architecture and custom loss function, we need to incorporate the two inputs and custom loss into the train_step method. Here’s how you can define and train such a model:

Define the model architecture to handle the two inputs.
Implement the train_step method to compute the custom loss.
Below is the complete implementation:

Explanation:
Model Architecture:

The ModelSubClassing class defines the layers in the __init__ method.
The call method specifies the forward pass using both inputs: the mel-spectrogram and the caption embedding.
Custom Training Step:

The train_step method is overridden to implement the custom loss function.
The custom loss combines the standard classification loss with the MSE between the caption embedding and the output of the dense layer of size 64.
Gradients are computed and applied to update the model’s weights.
Compilation and Training:

The model is compiled with an optimizer and loss function.
The model is trained using dummy data to illustrate the usage. In practice, you would replace the dummy data with your actual dataset.
This setup ensures that the model learns to minimize both the classification error and the difference between the caption embedding and the 64-dimensional dense layer output.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

class ModelSubClassing(keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        # Layer of Block 1
        self.conv1 = Conv2D(32, 3, strides=2, activation="relu")
        self.max1  = MaxPooling2D(3)
        self.bn1   = BatchNormalization()

        # Layer of Block 2
        self.conv2 = Conv2D(64, 3, activation="relu")
        self.bn2   = BatchNormalization()
        self.max2  = MaxPooling2D(3)
        self.drop1 = Dropout(0.3)
        
        # Layer of Block 3
        self.conv3 = Conv2D(128, 3, activation="relu")
        self.bn3   = BatchNormalization()
        self.max3  = MaxPooling2D(3)
        self.drop2 = Dropout(0.3)

        # Flatten, followed by 3 dense layers: 128, 64, num_classes
        self.flatten   = Flatten()
        self.dense128 = Dense(128, activation="relu")
        self.dense64 = Dense(64, activation="relu")
        self.dense_out = Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        # Unpack the inputs
        mel_spectrogram, caption_embedding = inputs

        # Forward pass: block 1
        x = self.conv1(mel_spectrogram)
        x = self.max1(x)
        x = self.bn1(x)

        # Forward pass: block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.max2(x)
        x = self.drop1(x, training=training)

        # Forward pass: block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.max3(x)
        x = self.drop2(x, training=training)

        # Flatten and dense layers
        x = self.flatten(x)
        x = self.dense128(x)
        x_dense64 = self.dense64(x)
        y_hat = self.dense_out(x_dense64)
        
        return y_hat, x_dense64

    def train_step(self, data):
        # Unpack the data
        (mel_spectrogram, caption_embedding), y = data

        with tf.GradientTape() as tape:
            y_pred, x_dense64 = self((mel_spectrogram, caption_embedding), training=True)  # Forward pass
            # Compute the standard loss value
            loss = self.compiled_loss(y, y_pred)
            # Add the custom loss component
            custom_loss = tf.reduce_mean(tf.square(caption_embedding - x_dense64))
            total_loss = loss + custom_loss

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

# Example of using the custom model
num_classes = 10  # Example number of output classes
model = ModelSubClassing(num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create some dummy data
import numpy as np
X_mel = np.random.random((10, 128, 1255, 1))
X_caption = np.random.random((10, 64))
y = np.random.randint(0, num_classes, 10)

# Train the model
model.fit((X_mel, X_caption), y, epochs=3, batch_size=2)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
# from tensorflow.keras import Model

# # Define a custom model class
# class MyCustomModel(Model):
#     def __init__(self):
#         super(MyCustomModel, self).__init__()
#         # Initialize layers
#         self.conv1 = Conv2D(32, (3, 3), activation='relu')
#         self.pool1 = MaxPooling2D((2, 2))
#         self.conv2 = Conv2D(64, (3, 3), activation='relu')
#         self.pool2 = MaxPooling2D((2, 2))
#         self.flatten = Flatten()
#         self.dense1 = Dense(128, activation='relu')
#         self.dense2 = Dense(10, activation='softmax')
    
#     # Define the forward pass
#     def call(self, inputs):
#         x = self.conv1(inputs)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.pool2(x)
#         x = self.flatten(x)
#         x = self.dense1(x)
#         return self.dense2(x)

# # Instantiate the model
# model = MyCustomModel()

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Print model summary (note that a summary can be printed only after the model is built)
# input_shape = (None, 28, 28, 1)  # Example input shape
# model.build(input_shape)
# model.summary()
